# Machine Learning beadandó feladata
## Dataracing 2022: exportforgalom előrejelzés
Király Márk (AX83OL)

### Importáljuk a szükséges modulokat.

In [29]:
import pandas as pd
import optuna
from optuna.samplers import TPESampler, GPSampler
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from imblearn.over_sampling import SMOTE
import numpy as np

### Töltsük be az adatokat az alábbi sorrendben:
#### 1. csv betöltése, 2. célváltozó beállítása, 3. tanító-teszt 0,6-0,4 felosztás
Megj.: Ez a kommentek javítása miatt lett később lefuttatva, mint a többi cella => ha újból futtatnánk az összes cellát, akkor is hasonló MAE, MSE, R2 értékeket kapnánk.

In [49]:
# Adatok betöltése
df = pd.read_csv("https://raw.githubusercontent.com/karsarobert/Machine_Learning_2024/main/train.csv")

# Célváltozó és prediktorok beállítása
y = df['target_reg']
corr_col = ['arbevexp_2014', 'arbevexp_2015', 'arbevexp_2016', 'arbevert_2014', 'arbevert_2015', 'arbevert_2016', 'ranyag_2014', 'ranyag_2015', 'ranyag_2016', 'rszem_2016']
X = df[corr_col]

# Adatok felosztása tanító és teszt halmazokra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

### Végezzünk adatbővítést! (Zajjal és szintetikus adatok hozzáadásával is)
#### Zajt adunk hozzá 0,1-es faktorral és 2-szer annyi szintetikus adatot gyártunk lineáris kombinációval, mint az eredeti tanítóhalmaz
#### Ok: a tanítóhalmaz túl kicsi a 0,6-0,4-es felosztás miatt...

In [40]:
# Zaj hozzáadása
noise_factor = 0.1
X_train_noisy = X_train + noise_factor * np.random.randn(*X_train.shape)
X_test_noisy = X_test + noise_factor * np.random.randn(*X_test.shape)

# Szintetikus adatok generálása lineáris kombinációval
def generate_synthetic_data(X, y, num_samples):
    synthetic_X = []
    synthetic_y = []
    num_features = X.shape[1]
    
    for _ in range(num_samples):
        idx1, idx2 = np.random.choice(len(X), size=2, replace=False)
        alpha = np.random.rand()
        
        new_sample_X = alpha * X.iloc[idx1] + (1 - alpha) * X.iloc[idx2]
        new_sample_y = alpha * y.iloc[idx1] + (1 - alpha) * y.iloc[idx2]
        
        synthetic_X.append(new_sample_X)
        synthetic_y.append(new_sample_y)
    
    return pd.DataFrame(synthetic_X, columns=X.columns), pd.Series(synthetic_y)

# Generáljunk szintetikus adatokat a tanító halmazhoz
num_synthetic_samples = len(X_train) * 2  # 2-szer több szintetikus minta, mint az eredeti tanító halmaz
X_synthetic, y_synthetic = generate_synthetic_data(X_train, y_train, num_synthetic_samples)

# Kombináljuk az eredeti és szintetikus adatokat
X_train_augmented = pd.concat([X_train_noisy, X_synthetic])
y_train_augmented = pd.concat([y_train, y_synthetic])

# StandardScaler létrehozása és illesztése a tanító adatokra
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_augmented)
X_test_scaled = scaler.transform(X_test_noisy)

# Az újonnan zajosított és skálázott adatok ellenőrzése
print("X_train_scaled (augmented):", X_train_scaled[:5])
print("X_test_scaled:", X_test_scaled[:5])

X_train_scaled (augmented): [[-0.11353001 -0.11254884 -0.11248401 -0.00609474 -0.04272862 -0.19992831
   0.03284351 -0.03458288 -0.17977939 -0.17114873]
 [-0.11352997 -0.11254879 -0.11248395 -0.23174224 -0.23885215 -0.23578411
  -0.21773128 -0.22773672 -0.22175522 -0.18010741]
 [-0.11352999 -0.11254878 -0.1124841  -0.10233538 -0.1187236  -0.11997266
  -0.06456833 -0.07401679 -0.0566597  -0.16567638]
 [-0.11353006 -0.11254891 -0.11248407 -0.24466937 -0.24654198 -0.24253352
  -0.22689106 -0.23864735 -0.23098685 -0.18468017]
 [-0.11352995 -0.11254887 -0.11248399 -0.24172092 -0.24525681 -0.24252162
  -0.22748765 -0.23790381 -0.22894936 -0.17552071]]
X_test_scaled: [[-0.11353001 -0.11254883 -0.11248402 -0.23147299 -0.23756431 -0.23705724
  -0.21536593 -0.22864936 -0.2222903  -0.17987296]
 [-0.11352984 -0.11254879 -0.11248406  1.37234286  1.4793277   1.37297924
   0.90920258  1.09402199  0.95387358  2.63551017]
 [-0.11352989 -0.11254887 -0.11248408 -0.18424826 -0.19244134 -0.20253702
  -0.20

## Építsünk modelt!
### Futtatáskor figyeljünk rá, hogy a MAE-t és a további jellemzőket az eredeti halmazon mérjük (y_test, y_pred).
### A modell a következőket használja: 
#### Random Forest regresszió, 
#### Optuna hiperparaméterkereső keretrendszer, ezen belűl a TPESampler fa alapú keresőalgoritmus

In [41]:
# Célfüggvény definiálása
def objective(trial):
    # Hiperparaméterek meghatározása
    n_estimators = trial.suggest_int('n_estimators', 10, 200) #(10,200)
    max_depth = trial.suggest_int('max_depth', 1, 20) #(1,20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10) #(2,10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4) #(1,4)
    
    # Random Forest modell létrehozása a megadott hiperparaméterekkel
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
        n_jobs=-1
    )
    
    # Modell illesztése
    model.fit(X_train_scaled, y_train_augmented)
    
    # Előrejelzések készítése
    y_pred = model.predict(X_test_scaled)
    
    # MAE számolása
    mae = mean_absolute_error(y_test, y_pred)
    return mae

# "Tanulmány" létrehozása és futtatása
try:
    study = optuna.create_study(direction='minimize', sampler=TPESampler())
    study.optimize(objective, n_trials=100)  # , timeout=900) Tehát maximum 0,25 óráig fut - kikommentelve
except optuna.exceptions.TrialPruned as e:
    print("Optimization stopped:", e)

# Legjobb paraméterek kiíratása
print("Best hyperparameters: ", study.best_params)

# Legjobb modell létrehozása a legjobb paraméterekkel
best_params = study.best_params
best_model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42,  # Marad változatlan!
    n_jobs=-1  # Párhuzamosítás miatt kell (igaz csak CPU-n párhuzamosít...)
)

# Legjobb modell illesztése a teljes edzési adatokra
best_model.fit(X_train_scaled, y_train_augmented)

# Előrejelzések készítése a teszthalmazon
y_pred = best_model.predict(X_test_scaled)

# Hibametrikák kiértékelése és kiíratása
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('Final Model Performance on Test Set:')
print('Mean Squared Error:', mse)
print('R-squared Score:', r2)
print('Mean Absolute Error:', mae)

[I 2024-05-27 14:20:01,547] A new study created in memory with name: no-name-d5efb255-1b3c-4b10-9d3e-84a3dc49f2b5
[I 2024-05-27 14:20:02,453] Trial 0 finished with value: 52624.378209158735 and parameters: {'n_estimators': 15, 'max_depth': 7, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 0 with value: 52624.378209158735.
[I 2024-05-27 14:20:03,140] Trial 1 finished with value: 275591.2658043612 and parameters: {'n_estimators': 89, 'max_depth': 1, 'min_samples_split': 8, 'min_samples_leaf': 2}. Best is trial 0 with value: 52624.378209158735.
[I 2024-05-27 14:20:03,404] Trial 2 finished with value: 275286.1840526459 and parameters: {'n_estimators': 31, 'max_depth': 1, 'min_samples_split': 7, 'min_samples_leaf': 1}. Best is trial 0 with value: 52624.378209158735.
[I 2024-05-27 14:20:16,198] Trial 3 finished with value: 50732.13060849724 and parameters: {'n_estimators': 193, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 4}. Best is trial 3 with value: 50732.1

Best hyperparameters:  {'n_estimators': 125, 'max_depth': 12, 'min_samples_split': 7, 'min_samples_leaf': 1}
Final Model Performance on Test Set:
Mean Squared Error: 185225496112.8629
R-squared Score: 0.9698049442932114
Mean Absolute Error: 48889.61616231008


Best hyperparameters:  {'n_estimators': 125, 'max_depth': 12, 'min_samples_split': 7, 'min_samples_leaf': 1}
Final Model Performance on Test Set:
Mean Squared Error: 185225496112.8629
R-squared Score: 0.9698049442932114
Mean Absolute Error: 48889.61616231008

## MAE: 48889